### University of Virginia
### DS 5559: Big Data Analytics
### Linear Regression Modeling of California Home Prices
### Last updated: Oct 21, 2019

### Name: Jay Hombal
### Computing Id: mh4ey

**TOTAL POINTS: 10**

**Instructions**  
In this project, you will work with the California Home Price dataset to train a regression model and predict median home prices. Please do the following:  

1) (6 PTS) Go through all code and fill in the missing cells. This will prep data, train a model, predict, and evaluate model fit.  Compute and report the Mean Squared Error (MSE).  
2) (1 PT) Repeat Part 1 with at least one additional feature from the original set.  
3) (2 PTS) Repeat Part 1 with at least one engineered feature based on one or more variables from the original set.  
4) (1 PT) Repeat Part 1 using Lasso Regression

Please report resuts in the following way:  
In the **RESULTS SECTION** table at the very bottom, there are three cells where you should copy your code from parts 2,3,4.  
In the very last cell, print a dataframe containing two columns: `question_part` and `MSE`.  
This dataframe must report your MSE results.

**Data Source**  
StatLib---Datasets Archive  
http://lib.stat.cmu.edu/datasets/

In [1]:
import os
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.linalg import DenseVector
from pyspark.ml.feature import StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

spark = SparkSession.builder.getOrCreate()

In [2]:
# read text file into pyspark dataframe
filename = 'cal_housing_data_preproc_w_header.txt'
df = spark.read.csv(filename,  inferSchema=True, header = True)

In [3]:
df.show(5)

+------------------+-----------------+------------------+-----------+--------------+----------+----------+--------+---------+
|median_house_value|    median_income|housing_median_age|total_rooms|total_bedrooms|population|households|latitude|longitude|
+------------------+-----------------+------------------+-----------+--------------+----------+----------+--------+---------+
|          452600.0|           8.3252|              41.0|      880.0|         129.0|     322.0|     126.0|   37.88|  -122.23|
|          358500.0|           8.3014|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|   37.86|  -122.22|
|          352100.0|7.257399999999999|              52.0|     1467.0|         190.0|     496.0|     177.0|   37.85|  -122.24|
|          341300.0|           5.6431|              52.0|     1274.0|         235.0|     558.0|     219.0|   37.85|  -122.25|
|          342200.0|           3.8462|              52.0|     1627.0|         280.0|     565.0|     259.0|   37.85|  -

In [4]:
# Initialize the `standardScaler`
#stage 1 - scale the dataset
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled", 
                                withStd=True, withMean=False)
#
maxIter=10
lr = LinearRegression (labelCol='label',\
                       maxIter=maxIter)          
pipeline = Pipeline(stages=[standardScaler, lr])

paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.3])\
    .addGrid(lr.elasticNetParam, [0.8])\
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(),
                          numFolds=3)  # use 3+ folds in practice


#### Code for part 1

**Feature Engineering**

We want to do three more things before training a model:  

**SCALING (1 POINT)**   
Scale the response variable median_house_value, dividing by 100000 and saving into column median_house_value_final

In [5]:
df = df.withColumn('median_house_value_final', col('median_house_value')/100000)

**FEATURE ENGINEERING**  **(1 POINT)**  
Add new feature:  rooms_per_household

In [6]:
df = df.withColumn('rooms_per_household', col('total_rooms')/col('households'))

df.select('median_house_value_final','rooms_per_household').show(5)

+------------------------+-------------------+
|median_house_value_final|rooms_per_household|
+------------------------+-------------------+
|                   4.526|  6.984126984126984|
|                   3.585|  6.238137082601054|
|                   3.521|  8.288135593220339|
|                   3.413| 5.8173515981735155|
|                   3.422|  6.281853281853282|
+------------------------+-------------------+
only showing top 5 rows



**SELECT AND STANDARDIZE FEATURES**  **(2 POINTS)**

In [7]:
# retain these predictors for Part 1
vars_to_keep = ["median_house_value_final", 
              "total_bedrooms", 
              "population", 
              "households", 
              "median_income", 
              "rooms_per_household"]

# subset the dataframe on these predictors
df1 = df.select(vars_to_keep)

In [8]:
# extract labels and features; stored as RDDs
transformed_data = df1.rdd.map(lambda x: (x[0], DenseVector(x[1:])))
transformed_df = spark.createDataFrame(transformed_data, ['label', 'features'])
transformed_df.take(2)

[Row(label=4.526, features=DenseVector([129.0, 322.0, 126.0, 8.3252, 6.9841])),
 Row(label=3.585, features=DenseVector([1106.0, 2401.0, 1138.0, 8.3014, 6.2381]))]

In [9]:
seed = 314
train_test = [0.8, 0.2]
train_data, test_data = transformed_df.randomSplit(train_test,seed=seed)

In [10]:
lr_cv_model1 = crossval.fit(train_data)

In [11]:
prediction1 = lr_cv_model1.transform(test_data)

In [12]:
prediction1.show(5)

+-------+--------------------+--------------------+------------------+
|  label|            features|     features_scaled|        prediction|
+-------+--------------------+--------------------+------------------+
|0.14999|[267.0,628.0,225....|[0.62934545510244...|2.1614311926900918|
|  0.225|[73.0,216.0,63.0,...|[0.17206823304298...| 1.741170547626018|
|   0.25|[33.0,64.0,27.0,0...|[0.07778426973176...| 1.237948424939956|
|  0.342|[153.0,112.0,47.0...|[0.36063615966544...|1.2959688644113232|
|   0.35|[1747.0,6852.0,15...|[4.11785209761785...|1.6603959090871583|
+-------+--------------------+--------------------+------------------+
only showing top 5 rows



In [13]:
MSE1 = prediction1.select("prediction", "label")\
    .rdd\
    .map(lambda x: (x[0] - x[1])**2)\
    .reduce(lambda x,y : x+y) /prediction1.count()
MSE1

0.755384454564553

#### code for part 2

In [14]:
# Code for Part 2 - (1 PT) Repeat Part 1 with at least one additional feature from the original set.
# retain these predictors for Part 1
vars_to_keep2 = ["median_house_value_final", 
              "total_bedrooms", 
              "population", 
              "households", 
              "median_income",
              "rooms_per_household",
              "housing_median_age"]

# subset the dataframe on these predictors
df2 = df.select(vars_to_keep2)

df2.show(2)

+------------------------+--------------+----------+----------+-------------+-------------------+------------------+
|median_house_value_final|total_bedrooms|population|households|median_income|rooms_per_household|housing_median_age|
+------------------------+--------------+----------+----------+-------------+-------------------+------------------+
|                   4.526|         129.0|     322.0|     126.0|       8.3252|  6.984126984126984|              41.0|
|                   3.585|        1106.0|    2401.0|    1138.0|       8.3014|  6.238137082601054|              21.0|
+------------------------+--------------+----------+----------+-------------+-------------------+------------------+
only showing top 2 rows



In [15]:
# extract labels and features; stored as RDDs
transformed_data2 = df2.rdd.map(lambda x: (x[0], DenseVector(x[1:])))
transformed_df2 = spark.createDataFrame(transformed_data2, ['label', 'features'])
transformed_df2.take(2)

[Row(label=4.526, features=DenseVector([129.0, 322.0, 126.0, 8.3252, 6.9841, 41.0])),
 Row(label=3.585, features=DenseVector([1106.0, 2401.0, 1138.0, 8.3014, 6.2381, 21.0]))]

In [16]:
seed = 314
train_test = [0.8, 0.2]
train_data2, test_data2 = transformed_df2.randomSplit(train_test,seed=seed)

In [17]:
lr_cv_model2 = crossval.fit(train_data2)

In [18]:
prediction2 = lr_cv_model2.transform(test_data2)
prediction2.show(5)

+-------+--------------------+--------------------+------------------+
|  label|            features|     features_scaled|        prediction|
+-------+--------------------+--------------------+------------------+
|0.14999|[267.0,628.0,225....|[0.62934545510244...|2.1614311751602564|
|  0.225|[73.0,216.0,63.0,...|[0.17206823304298...|1.7411706113489145|
|   0.25|[33.0,64.0,27.0,0...|[0.07778426973176...|1.2379485859552657|
|  0.342|[153.0,112.0,47.0...|[0.36063615966544...| 1.295969014209025|
|   0.35|[1747.0,6852.0,15...|[4.11785209761785...| 1.660395988426935|
+-------+--------------------+--------------------+------------------+
only showing top 5 rows



In [19]:
MSE2 = prediction2.select("prediction", "label")\
    .rdd\
    .map(lambda x: (x[0] - x[1])**2)\
    .reduce(lambda x,y : x+y) /prediction2.count()
MSE2

0.7553845110003091

#### Code for part 3

In [20]:
# Code for Part 3 - Repeat Part 1 with at least one engineered feature based on one or more variables from the original set.
df3 = df.withColumn('median_house_value_final', col('median_house_value')/100000)

# add rooms_per_household 
df = df.withColumn('rooms_per_household', col('total_rooms')/col('households'))

# add population_per_household (num people in the home)
df3 = df.withColumn('population_per_household', col('population')/col('households'))


df3.select('median_house_value_final','rooms_per_household','population_per_household').show(2)

print(df3.columns)

+------------------------+-------------------+------------------------+
|median_house_value_final|rooms_per_household|population_per_household|
+------------------------+-------------------+------------------------+
|                   4.526|  6.984126984126984|      2.5555555555555554|
|                   3.585|  6.238137082601054|       2.109841827768014|
+------------------------+-------------------+------------------------+
only showing top 2 rows

['median_house_value', 'median_income', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'latitude', 'longitude', 'median_house_value_final', 'rooms_per_household', 'population_per_household']


In [21]:
# retain these predictors for Part 1
vars_to_keep2 = ["median_house_value_final", 
              "total_bedrooms", 
              "population", 
              "households", 
              "median_income",
              "rooms_per_household",
              "population_per_household"]

# subset the dataframe on these predictors
df3 = df3.select(vars_to_keep2)

df3.show(2)

+------------------------+--------------+----------+----------+-------------+-------------------+------------------------+
|median_house_value_final|total_bedrooms|population|households|median_income|rooms_per_household|population_per_household|
+------------------------+--------------+----------+----------+-------------+-------------------+------------------------+
|                   4.526|         129.0|     322.0|     126.0|       8.3252|  6.984126984126984|      2.5555555555555554|
|                   3.585|        1106.0|    2401.0|    1138.0|       8.3014|  6.238137082601054|       2.109841827768014|
+------------------------+--------------+----------+----------+-------------+-------------------+------------------------+
only showing top 2 rows



In [22]:
# extract labels and features; stored as RDDs
transformed_data3 = df3.rdd.map(lambda x: (x[0], DenseVector(x[1:])))
transformed_df3 = spark.createDataFrame(transformed_data3, ['label', 'features'])
seed = 314
train_test = [0.8, 0.2]
train_data3, test_data3 = transformed_df.randomSplit(train_test,seed=seed)
transformed_df3.take(2)

[Row(label=4.526, features=DenseVector([129.0, 322.0, 126.0, 8.3252, 6.9841, 2.5556])),
 Row(label=3.585, features=DenseVector([1106.0, 2401.0, 1138.0, 8.3014, 6.2381, 2.1098]))]

In [23]:
lr_cv_model3 = crossval.fit(train_data3)

In [24]:
prediction3 = lr_cv_model3.transform(test_data3)

In [25]:
prediction3.show(5)

+-------+--------------------+--------------------+------------------+
|  label|            features|     features_scaled|        prediction|
+-------+--------------------+--------------------+------------------+
|0.14999|[267.0,628.0,225....|[0.62934545510244...|2.1614311926900918|
|  0.225|[73.0,216.0,63.0,...|[0.17206823304298...| 1.741170547626018|
|   0.25|[33.0,64.0,27.0,0...|[0.07778426973176...| 1.237948424939956|
|  0.342|[153.0,112.0,47.0...|[0.36063615966544...|1.2959688644113232|
|   0.35|[1747.0,6852.0,15...|[4.11785209761785...|1.6603959090871583|
+-------+--------------------+--------------------+------------------+
only showing top 5 rows



In [26]:
MSE3 = prediction3.select("prediction", "label")\
    .rdd\
    .map(lambda x: (x[0] - x[1])**2)\
    .reduce(lambda x,y : x+y) /prediction3.count()
MSE3

0.755384454564553

#### Code for Part 4

In [27]:
# setting parameters for Lasso regiression
# elasticNetParam corresponds to α and regParam corresponds to λ
# Lasso - When λ>0 (i.e. regParam >0) and α = 1 (i.e. elasticNetParam =1), then the penalty is an L1 penalty.

paramGrid2 = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1])\
    .addGrid(lr.elasticNetParam, [1.0])\
    .build()

crossval2 = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid2,
                          evaluator=RegressionEvaluator(),
                          numFolds=3)  # use 3+ folds in practice

In [28]:
lr_cv_model4 = crossval2.fit(train_data)

In [29]:
prediction4 = lr_cv_model4.transform(test_data)

In [30]:
prediction4.show(5)

+-------+--------------------+--------------------+------------------+
|  label|            features|     features_scaled|        prediction|
+-------+--------------------+--------------------+------------------+
|0.14999|[267.0,628.0,225....|[0.62934545510244...| 2.190421297020153|
|  0.225|[73.0,216.0,63.0,...|[0.17206823304298...|1.6357883023074404|
|   0.25|[33.0,64.0,27.0,0...|[0.07778426973176...|0.9716680802759292|
|  0.342|[153.0,112.0,47.0...|[0.36063615966544...|1.0482397280771312|
|   0.35|[1747.0,6852.0,15...|[4.11785209761785...|1.5291871246910729|
+-------+--------------------+--------------------+------------------+
only showing top 5 rows



In [31]:
MSE4 = prediction4.select("prediction", "label")\
    .rdd\
    .map(lambda x: (x[0] - x[1])**2)\
    .reduce(lambda x,y : x+y) /prediction4.count()
MSE4

0.691575957689366

#### MSE for Question part 1 to part 4

Print dataframe containing `question_part`, `MSE` values for parts 1-4 in the next cell.

In [32]:
# print dataframe containing question_part, MSE
mse_dict = {
        'Question part' :  ['Part 1',
                            'Part 2',
                            'Part 3',
                            'Part 4'],
                           
         'MSE' : [MSE1, MSE2, MSE3, MSE4]
        }
pd.set_option("display.max_columns", 100)
pd.set_option("max_colwidth", 80)
mse_df = pd.DataFrame(mse_dict)
mse_df.head()

,Question part,MSE
0,Part 1,0.755384
1,Part 2,0.755385
2,Part 3,0.755384
3,Part 4,0.691576


In [33]:
!jupyter nbconvert DS5559_M54HW_calhousing_pipeline_JayHombal.ipynb --to pdf

[NbConvertApp] Converting notebook DS5559_M54HW_calhousing_pipeline_JayHombal.ipynb to pdf
[NbConvertApp] Writing 61121 bytes to DS5559_M54HW_calhousing_pipeline_JayHombal.pdf
